# Libraries

In [1]:
# Libraries
import os
import pandas as pd
import numpy as np
import sklearn
import itertools
import json
import openai
import tiktoken
import spacy

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import OpenAI




/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Plotly Functions for IDP

In [2]:
# import numpy as np
# import pandas as pd
# from umap import UMAP
# from typing import List, Union
# from sklearn.preprocessing import MinMaxScaler

# import plotly.express as px
# import plotly.graph_objects as go


# def visualize_topics(topic_model,
#                      topics: List[int] = None,
#                      top_n_topics: int = None,
#                      custom_labels: Union[bool, str] = False,
#                      title: str = "<b>Intertopic Distance Map</b>",
#                      width: int = 650,
#                      height: int = 650) -> go.Figure:
#     """ Visualize topics, their sizes, and their corresponding words

#     This visualization is highly inspired by LDAvis, a great visualization
#     technique typically reserved for LDA.

#     Arguments:
#         topic_model: A fitted BERTopic instance.
#         topics: A selection of topics to visualize
#         top_n_topics: Only select the top n most frequent topics
#         custom_labels: If bool, whether to use custom topic labels that were defined using 
#                        `topic_model.set_topic_labels`.
#                        If `str`, it uses labels from other aspects, e.g., "Aspect1".
#         title: Title of the plot.
#         width: The width of the figure.
#         height: The height of the figure.

#     Examples:

#     To visualize the topics simply run:

#     ```python
#     topic_model.visualize_topics()
#     ```

#     Or if you want to save the resulting figure:

#     ```python
#     fig = topic_model.visualize_topics()
#     fig.write_html("path/to/file.html")
#     ```
#     <iframe src="../../getting_started/visualization/viz.html"
#     style="width:1000px; height: 680px; border: 0px;""></iframe>
#     """
#     # Select topics based on top_n and topics args
#     freq_df = topic_model.get_topic_freq()
#     freq_df = freq_df.loc[freq_df.Topic != -1, :]
#     if topics is not None:
#         topics = list(topics)
#     elif top_n_topics is not None:
#         topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
#     else:
#         topics = sorted(freq_df.Topic.to_list())

#     # Extract topic words and their frequencies
#     topic_list = sorted(topics)
#     frequencies = [topic_model.topic_sizes_[topic] for topic in topic_list]
#     if isinstance(custom_labels, str):
#         words = [[[str(topic), None]] + topic_model.topic_aspects_[custom_labels][topic] for topic in topic_list]
#         words = ["_".join([label[0] for label in labels[:4]]) for labels in words]
#         words = [label if len(label) < 30 else label[:27] + "..." for label in words]
#     elif custom_labels and topic_model.custom_labels_ is not None:
#         words = [topic_model.custom_labels_[topic + topic_model._outliers] for topic in topic_list]
#     else:
#         words = [" | ".join([word[0] for word in topic_model.get_topic(topic)[:5]]) for topic in topic_list]

#     # Embed c-TF-IDF into 2D
#     all_topics = sorted(list(topic_model.get_topics().keys()))
#     indices = np.array([all_topics.index(topic) for topic in topics])

#     if topic_model.topic_embeddings_ is not None:
#         embeddings = topic_model.topic_embeddings_[indices]
#         embeddings = UMAP(n_neighbors=2, n_components=2, metric='cosine', random_state=42).fit_transform(embeddings)
#     else:
#         embeddings = topic_model.c_tf_idf_.toarray()[indices]
#         embeddings = MinMaxScaler().fit_transform(embeddings)
#         embeddings = UMAP(n_neighbors=2, n_components=2, metric='hellinger', random_state=42).fit_transform(embeddings)

#     # Visualize with plotly
#     df = pd.DataFrame({"x": embeddings[:, 0], "y": embeddings[:, 1],
#                        "Topic": topic_list, "Words": words, "Size": frequencies})
#     return _plotly_topic_visualization(df, topic_list, title, width, height)


# def _plotly_topic_visualization(df: pd.DataFrame,
#                                 topic_list: List[str],
#                                 title: str,
#                                 width: int,
#                                 height: int):
#     """ Create plotly-based visualization of topics with a slider for topic selection """

#     def get_color(topic_selected):
#         if topic_selected == -1:
#             marker_color = ["#B0BEC5" for _ in topic_list]
#         else:
#             marker_color = ["red" if topic == topic_selected else "#B0BEC5" for topic in topic_list]
#         return [{'marker.color': [marker_color]}]

#     # Prepare figure range
#     x_range = (df.x.min() - abs((df.x.min()) * .15), df.x.max() + abs((df.x.max()) * .15))
#     y_range = (df.y.min() - abs((df.y.min()) * .15), df.y.max() + abs((df.y.max()) * .15))

#     # Plot topics
#     fig = px.scatter(df, x="x", y="y", size="Size", text = "Topic", size_max=40, template="simple_white", labels={"x": "", "y": ""} #,
#                      #hover_data={"Topic": True, "Words": True, "Size": True, "x": False, "y": False}
#                      )
#     #fig.update_traces(marker=dict(color="#B0BEC5", line=dict(width=2, color='DarkSlateGrey')))

#     # Update hover order
#     # fig.update_traces(hovertemplate="<br>".join(["<b>Topic %{customdata[0]}</b>",
#     #                                              "%{customdata[1]}",
#     #                                              "Size: %{customdata[2]}"]))

#     # Create a slider for topic selection
#     #steps = [dict(label=f"Topic {topic}", method="update", args=get_color(topic)) for topic in topic_list]
#     #sliders = [dict(active=0, pad={"t": 50}, steps=steps)]

#     # Stylize layout
#     # fig.update_layout(
#     #     title={
#     #         'text': f"{title}",
#     #         'y': .95,
#     #         'x': 0.5,
#     #         'xanchor': 'center',
#     #         'yanchor': 'top',
#     #         'font': dict(
#     #             size=22,
#     #             color="Black")
#     #     },
#     #     width=width,
#     #     height=height,
#     #     #hoverlabel=dict(
#     #         #bgcolor="white",
#     #         #font_size=16,
#     #         #font_family="Rockwell"
#     #    # ),
#     #     xaxis={"visible": False},
#     #     yaxis={"visible": False},
#     #    #sliders=sliders
#     # )

#     # # Update axes ranges
#     # fig.update_xaxes(range=x_range)
#     # fig.update_yaxes(range=y_range)

#     # Add grid in a 'plus' shape
#     # fig.add_shape(type="line",
#     #               x0=sum(x_range) / 2, y0=y_range[0], x1=sum(x_range) / 2, y1=y_range[1],
#     #               line=dict(color="#CFD8DC", width=2))
#     # fig.add_shape(type="line",
#     #               x0=x_range[0], y0=sum(y_range) / 2, x1=x_range[1], y1=sum(y_range) / 2,
#     #               line=dict(color="#9E9E9E", width=2))
#     # fig.add_annotation(x=x_range[0], y=sum(y_range) / 2, text="D1", showarrow=False, yshift=10)
#     # fig.add_annotation(y=y_range[1], x=sum(x_range) / 2, text="D2", showarrow=False, xshift=10)
#     # fig.data = fig.data[::-1]

#     return fig

## Data Loading and Code Pairing

In [3]:
# Load the dataset to check its structure
file_path = 'AI_interview_data.csv'

# read data
df = pd.read_csv(file_path)

# Display the first few rows of the dataset and the column names
df.head(), df.columns

(   ID Interview.ID          Role           Speaker  \
 0   1           AH    Researcher      Researcher 1   
 1   2           AH  Grad Student  Allyson Hauptman   
 2   3           AH    Researcher      Researcher 1   
 3   4           AH  Grad Student  Allyson Hauptman   
 4   5           AH    Researcher      Researcher 1   
 
                                            Utterance  Line  Stanza  \
 0  Okay, here we go. All right. So I'm recording ...     1       1   
 1                                               Yes.     2       1   
 2  Okay, great. And what is the date today? Today...     3       1   
 3  Sure. So I am an active duty army officer. I w...     4       1   
 4  Okay. Can you tell us more about, you said the...     5       1   
 
    ML.AI.Applications  Limitations.of.ML.AI  Privacy  \
 0                   0                     0        0   
 1                   0                     0        0   
 2                   0                     0        0   
 3          

In [4]:
# Pairing Codes for Connection
# I assume the codes columns will ALWAYS begin immediately after the Stanza col
code_columns = df.columns[df.columns.get_loc('Stanza') + 1:]

# Generating all combinations of code pairs
code_pairs = list(itertools.combinations(code_columns, 2))

# Display the code columns and the first few pairs to verify
len(code_pairs), code_columns, code_pairs[:5], print(f"There are {len(code_pairs)} possible pairs of codes or links/connections to be examined in the coded data")

There are 45 possible pairs of codes or links/connections to be examined in the coded data


(45,
 Index(['ML.AI.Applications', 'Limitations.of.ML.AI', 'Privacy',
        'Human.involvemnt.interaction', 'Bias..Prejudice..and.Harm',
        'Minimizing.Bias.and.Harm', 'Assessing.Underlying.Motivations',
        'Trust.Building', 'Algorithms.and.computational.thinking',
        'Training.and.Testing.Machines.Using.Data'],
       dtype='object'),
 [('ML.AI.Applications', 'Limitations.of.ML.AI'),
  ('ML.AI.Applications', 'Privacy'),
  ('ML.AI.Applications', 'Human.involvemnt.interaction'),
  ('ML.AI.Applications', 'Bias..Prejudice..and.Harm'),
  ('ML.AI.Applications', 'Minimizing.Bias.and.Harm')],
 None)

## SEGMENTATION 1A: WHOLE WINDOW (without referent line logic)

In [5]:
# Algorithm for whole data
def find_cooccurrences_in_windows(df, code_pairs, window_size):
    pair_dfs = {}
    
    # Iterate through each code pair
    for pair in code_pairs:
        code1, code2 = pair
        pair_key = f"{code1}_{code2}"
        pair_dfs[pair_key] = []

        # Slide through the dataframe using the specified window size
        for start in range(len(df) - window_size + 1):
            window = df.iloc[start:start + window_size]
            
            # Check if there are any lines where both codes are coded as 1
            co_occurrences = window[(window[code1] == 1) & (window[code2] == 1)]
            
            if not co_occurrences.empty:
                # Concatenate all the Utterances in the window
                concatenated_utterances = window['Utterance'].tolist()
                pair_dfs[pair_key].append(concatenated_utterances)

    return pair_dfs

In [6]:
# applying
window_size = 4  # Adjust the window size as needed
pair_occurrences_in_windows = find_cooccurrences_in_windows(df, code_pairs, window_size)

In [7]:
# Checking if it works
#for key, value in pair_occurrences_in_windows.items():
    #print(f"{key}: {value[:7]}")  # Printing only first 7 windows for brevity

#### Padding & Saving

In [8]:
# function for saving paired files for the list of list ones
def save_pairs_to_csv(pair_dfs, output_file):
    # Convert the dictionary to a DataFrame
    max_len = max(len(v) for v in pair_dfs.values())  # Find the maximum length of lists in the dictionary
    data = {k: v + [None] * (max_len - len(v)) for k, v in pair_dfs.items()}  # Pad lists with None
    df_output = pd.DataFrame(data)
    
    # Save the DataFrame to a CSV file
    df_output.to_csv(output_file, index=False)

In [9]:
# Saving
whole_output_file = 'whole_paired_code_occurrences.csv'
save_pairs_to_csv(pair_occurrences_in_windows, whole_output_file)

print(f"Data saved to {whole_output_file}")

Data saved to whole_paired_code_occurrences.csv


## SEGMENTATION 1B: WHOLE WINDOW (WITH REFERENT LINE LOGIC)

This function traverses the data using a sliding window. It uses the last line of each window as the referent line. If there is a co-occurrence in the referent line, it concatenates all the `Utterances` in the window (including the referent line) and save them to a list specific to that code pair.

In [10]:
def find_win_cooccurrences_with_referent(df, code_pairs, window_size):
    pair_dfs = {}
    
    # Iterate through each code pair
    for pair in code_pairs:
        code1, code2 = pair
        pair_key = f"{code1}_{code2}"
        pair_dfs[pair_key] = []

        # Slide through the dataframe using the specified window size
        for start in range(len(df) - window_size + 1):
            window = df.iloc[start:start + window_size]
            referent_line = window.iloc[-1]
            
            # Check if there is a co-occurrence in the referent line
            if referent_line[code1] == 1 and referent_line[code2] == 1:
                # Concatenate all the Utterances in the window
                concatenated_utterances = window['Utterance'].tolist()
                pair_dfs[pair_key].append(concatenated_utterances)

    return pair_dfs

In [11]:
# applying
window_size = 4  # Adjust the window size as needed
pair_win_occurrences_with_referent = find_win_cooccurrences_with_referent(df, code_pairs, window_size)


In [12]:
# Checking if it works
#for key, value in pair_win_occurrences_with_referent.items():
    #print(f"{key}: {value[:7]}")  # Printing only first 7 windows for brevity

In [13]:
# Save the results to a CSV file
whole_ref_output_file = 'paired_whole_occurrences_window_referent.csv'
save_pairs_to_csv(pair_win_occurrences_with_referent, whole_ref_output_file)


## SEGMENTATION 2: ONLY CODED LINES

The function below performs the following actions:
1. For each window, set the referent line as the last line in the window.
2. Check if any of the coded pairs is coded as 1 in the referent line.
3. If a code is coded as 1 in the referent line, then


a. It check if there's a co-occurrence with the other code in the same line. If true, save the referent line's utterance as a list.


b. It check if any other line in the window has the second code coded as 1. If true, concatenate the referent line with this line and save them as a list.

In [14]:
def find_code_occurrences_with_referent(df, code_pairs, window_size):
    pair_dfs = {}
    
    # Iterate through each code pair
    for pair in code_pairs:
        code1, code2 = pair
        pair_key = f"{code1}_{code2}"
        pair_dfs[pair_key] = []

        # Slide through the dataframe using the specified window size
        for start in range(len(df) - window_size + 1):
            window = df.iloc[start:start + window_size]
            referent_line = window.iloc[-1]
            
            # Check if any of the code pairs is coded as 1 in the referent line
            if referent_line[code1] == 1 or referent_line[code2] == 1:
                # Check for co-occurrence of both codes in the referent line
                if referent_line[code1] == 1 and referent_line[code2] == 1:
                    pair_dfs[pair_key].append([referent_line['Utterance']])
                
                # Check for co-occurrence with any other line in the window, excluding the referent line itself
                for i in range(len(window) - 1):
                    row = window.iloc[i]
                    if row[code1] == 1 and referent_line[code2] == 1:
                        concatenated_lines = f"{row['Utterance']} | {referent_line['Utterance']}"
                        # if you prefer that each utterance be within its own quote in the list, uncomment code below
                        # concatenated_lines = [f'"{row["Utterance"]}"', f'"{referent_line["Utterance"]}"']
                        pair_dfs[pair_key].append([concatenated_lines])
                    elif row[code2] == 1 and referent_line[code1] == 1:
                        concatenated_lines = f"{row['Utterance']} | {referent_line['Utterance']}"
                        # if you prefer that each utterance be within its own quote in the list, uncomment code below
                        # concatenated_lines = [f'"{row["Utterance"]}"', f'"{referent_line["Utterance"]}"']
                        pair_dfs[pair_key].append([concatenated_lines])

    return pair_dfs

In [15]:
# applying function to data
window_size = 4  # You can adjust the window size as needed
pair_occurrences_with_referent = find_code_occurrences_with_referent(df, code_pairs, window_size)


In [16]:
# Checking if it works
#for key, value in pair_occurrences_with_referent.items():
    #print(f"{key}: {value[:7]}")  # Printing only first 7 windows for brevity

In [17]:
# Saving
output_file = 'ref_pipe_paired_code_occurrences.csv'
save_pairs_to_csv(pair_occurrences_with_referent, output_file)


## Adding the Speaker Column For Tracking

In [18]:
## ------------------------------This function includes the Speaker In The Segmentation ---------------------------##

def find_occurrences_with_referent(df, code_pairs, window_size):
    pair_dfs = {}
    
    # Iterate through each code pair
    for pair in code_pairs:
        code1, code2 = pair
        pair_key = f"{code1}_{code2}"
        pair_dfs[pair_key] = []

        # Slide through the dataframe using the specified window size
        for start in range(len(df) - window_size + 1):
            window = df.iloc[start:start + window_size]
            referent_line = window.iloc[-1]
            
            # Check if any of the code pairs is coded as 1 in the referent line
            if referent_line[code1] == 1 or referent_line[code2] == 1:
                # Check for co-occurrence of both codes in the referent line
                if referent_line[code1] == 1 and referent_line[code2] == 1:
                    pair_dfs[pair_key].append([f"{referent_line['Speaker']}: {referent_line['Utterance']}"])
                
                # Check for co-occurrence with any other line in the window, excluding the referent line itself
                for i in range(len(window) - 1):
                    row = window.iloc[i]
                    if row[code1] == 1 and referent_line[code2] == 1:
                        concatenated_lines = f"{row['Speaker']}: {row['Utterance']} | {referent_line['Speaker']}: {referent_line['Utterance']}"
                        pair_dfs[pair_key].append([concatenated_lines])
                    elif row[code2] == 1 and referent_line[code1] == 1:
                        concatenated_lines = f"{row['Speaker']}: {row['Utterance']} | {referent_line['Speaker']}: {referent_line['Utterance']}"
                        pair_dfs[pair_key].append([concatenated_lines])

    return pair_dfs


##-----------------------------------------------------------------------------------------------------------------##

In [19]:
# apply the algorithm
window_size = 4  # You can adjust the window size as needed
code_occurrences_with_referent = find_occurrences_with_referent(df, code_pairs, window_size)

In [20]:
# Checking if it works
#for key, value in code_occurrences_with_referent.items():
    #print(f"{key}: {value[:7]}")  # Printing only first 7 windows for brevity

# prepare data for BERT

In [21]:
## Convert csv to list function because I can't figure out to index the dictionary!!!! :o
import csv
def column_to_list(csv_file, column_index):
    data = []
    with open(csv_file, 'r', newline='', encoding="cp1252") as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) > column_index:  # Check if the row has the desired column
                data.append(row[column_index])
   # del data[0]
    return data

In [22]:
## Create a list of documents for topic modeling from csv file

# #coded lines only 
# csv_file = "ref_pipe_paired_code_occurrences.csv" 
# column_index = 33 #select column/code pair 
# docs_clo = column_to_list(csv_file, column_index)
# print("code pair for clo is ", docs_clo[0])
# del docs_clo[0]
# docs_clo = list(filter(None, docs_clo))


# #whole window
# csv_file = "paired_whole_occurrences_window_referent.csv" 
# column_index = 2 #select column/code pair 
# docs_ww = column_to_list(csv_file, column_index)
# print("code pair for ww is ", docs_ww[0])
# del docs_ww[0]
# docs_ww = list(filter(None, docs_ww))

# # check output
# #docs_ww
# #docs_clo


# PCA instead of UMAP (to remove stochastic behavior/randomness in dim red)

In [87]:
#select column/code pair for coded lines only list
column_index =  28

#create list for BERTopic
csv_file = "ref_pipe_paired_code_occurrences.csv" 
docs_clo = column_to_list(csv_file, column_index)
print("code pair for clo is ", docs_clo[0])
del docs_clo[0]
docs_clo = list(filter(None, docs_clo))


code pair for clo is  Human.involvemnt.interaction_Algorithms.and.computational.thinking


In [88]:
# 1. Extract Embeddings
#embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5") 
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

# 2. Reduce Dimensionality
pca_model = PCA(n_components=30)

# 3. Cluster Reduced Embeddings
hdbscan_model = HDBSCAN(min_cluster_size=4, metric='euclidean', prediction_data=True)

# 4. Tokenize Topics
vectorizer_model = CountVectorizer(stop_words="english")

# 5. Create Topic Reprensetations
ctfidf_model = ClassTfidfTransformer()

# 6. Fine-tune representations
#key_representation_model = KeyBERTInspired()'
key_representation_model = MaximalMarginalRelevance()

# Create model
alt_topic_model = BERTopic(
    embedding_model = embedding_model,
    umap_model=pca_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    representation_model=key_representation_model, 
    calculate_probabilities=True
    )



In [89]:
alt_topics, alt_probs = alt_topic_model.fit_transform(docs_clo)


In [90]:
these_docs = alt_topic_model.get_document_info(docs_clo)
these_docs.to_csv("results/docs_pair3.csv")

In [77]:
these_topics = alt_topic_model.get_topic_info()
these_topics.to_csv("results/topics_pair1.csv", index=False)

these_topics

,Topic,Count,Name,Representation,Representative_Docs
0,-1,152,-1_like_ai_data_learning,"[like, ai, data, learning, things, machine, pe...","[['Yeah. I love that, Joe. And I can see that ..."
1,0,13,0_computer_ai_school_tech,"[computer, ai, school, tech, degree, career, g...","[[""Yeah. So pretty much been an academic in tr..."
2,1,8,1_ai_teammates_collaboration_humans,"[ai, teammates, collaboration, humans, team, t...","[[""So a lot of times human... we would call it..."
3,2,8,2_robot_text_speech_work,"[robot, text, speech, work, techniques, needs,...",[['A bit. Yeah. So the primary ones that I\'d ...
4,3,8,3_ai_ethics_students_case,"[ai, ethics, students, case, focus, promises, ...",[['I know that there\'s a K12 curriculum that ...
5,4,7,4_logical_teaching_language_kids,"[logical, teaching, language, kids, logic, cod...","[[""I think it's super important because it tea..."
6,5,7,5_game_kids_information_hacked,"[game, kids, information, hacked, risk, unethi...","[[""I would say one way is to maybe if they hav..."
7,6,6,6_implemented_mimic_capabilities_ai,"[implemented, mimic, capabilities, ai, ability...","[[""That's interesting how that seems very appl..."
8,7,6,7_models_reinforcement_training_kids,"[models, reinforcement, training, kids, learni...",[['Hmm. I think kids would probably always lik...
9,8,5,8_security_relationship_tools_capabilities,"[security, relationship, tools, capabilities, ...","[[""Sure. So the big differentiator for what wo..."


In [82]:
#topic_distr, _ = alt_topic_model.approximate_distribution(docs_clo, min_similarity=0)
#alt_topic_model.visualize_distribution(topic_distr[131])
alt_topic_model.visualize_distribution(alt_probs[131])

In [27]:
alt_topic_model.visualize_barchart()

In [28]:
# get idp coordinates from plot

idp = alt_topic_model.visualize_topics()
#print(idp.show)
topic_data = idp.data

dots = topic_data[0]["marker"]
dots = np.array(dots.size)

x_coord = np.array((topic_data[0]['x']))
y_coord = np.array((topic_data[0]['y']))

topic_nums = these_topics.Topic
topic_nums = topic_nums.drop(topic_nums.index[[0]])

topic_labels = these_topics.Name
topic_labels = topic_labels.drop(topic_labels.index[[0]])

idp_coordinates = pd.DataFrame({"x":x_coord, "y":y_coord, "topic":topic_labels, "dot_size":dots, "topics_nums":topic_nums})
#print(idp_coordinates)
#idp_coordinates.to_csv("idpCoords.csv")

# plot IDP on my own

import plotly.express as px

def improve_text_position(x):
    # fix indentation 
    positions = [ "middle left" , "middle right" ] 
    return [positions[i % len(positions)] for i in range(len(x))]

df = idp_coordinates

fig = px.scatter(df, 
                 x="x", 
                 y="y", 
                 text="topics_nums", 
                 color = "topic",
                 size ="dot_size",
                 labels=dict(x="D1", y="D2"),
                 width = 1000,
                 height = 700,
                 title= "Intertopic Distance Map"
                 )
fig.update_traces(
   hovertemplate=None,
   hoverinfo='skip',
   textposition=improve_text_position(df['x']
   )
)

fig.update_layout(
     xaxis={"visible": False},
    yaxis={"visible": False}
)
   
# Prepare figure range
x_range = (df.x.min() - abs((df.x.min()) * .15), df.x.max() + abs((df.x.max()) * .15))
y_range = (df.y.min() - abs((df.y.min()) * .15), df.y.max() + abs((df.y.max()) * .15))

# Update axes ranges
fig.update_xaxes(range=x_range)
fig.update_yaxes(range=y_range)

#fig.write_image("results/idp_pair1.png")

fig.show()


In [29]:
alt_topic_model.visualize_documents(docs_clo)

# BERT

In [30]:
# Regular ol' BERT
representation_model = BERTopic()
topic_model = BERTopic(representation_model=representation_model)

In [31]:
# run and fit BERT
topics, probs = topic_model.fit_transform(docs_clo)



In [32]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,38,-1_the_that_and_to,"[the, that, and, to, it, of, so, is, are, you]",[['A bit. Yeah. So the primary ones that I\'d ...
1,0,198,0_and_the_to_that,"[and, the, to, that, of, it, in, so, is, you]","[[""Yeah. So pretty much been an academic in tr..."
2,1,12,1_of_and_was_that,"[of, and, was, that, the, or, gaze, it, kind, ...","[[""Yeah. Yeah. It's cool. And what's interesti..."


# keyBERT

In [33]:
# Fine tuning the model with KeyBERTInspired
#embedding_model = SentenceTransformer("all-mpnet-base-v2")
#hbdscan_model = KMeans(n_clusters=5)
key_representation_model = KeyBERTInspired()
umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=8, metric='euclidean', prediction_data=True)

key_topic_model = BERTopic(representation_model=key_representation_model, umap_model=umap_model, hdbscan_model=hdbscan_model)

In [34]:
# run and fit keyBERT
key_topics, key_probs = key_topic_model.fit_transform(docs_clo)



In [35]:
# Frequent topic using keyBERT
key_topic_model.get_topic_info()



,Topic,Count,Name,Representation,Representative_Docs
0,-1,87,-1_ai_robots_learning_robot,"[ai, robots, learning, robot, machine, data, h...",[['A bit. Yeah. So the primary ones that I\'d ...
1,0,51,0_programming_ai_kids_teaching,"[programming, ai, kids, teaching, learning, yo...",[['Yeah. That makes a lot of sense. What do yo...
2,1,27,1_ai_teammates_teaming_collaboration,"[ai, teammates, teaming, collaboration, resear...","[[""So a lot of times human... we would call it..."
3,2,22,2_ai_ethics_moral_questions,"[ai, ethics, moral, questions, students, do, d...","[['Okay, here\'s how I\'m going to think about..."
4,3,15,3_ai_intelligence_intelligent_artificial,"[ai, intelligence, intelligent, artificial, le...","[[""It'll be interesting to see what type of ad..."
5,4,14,4_ai_privacy_kids_intelligence,"[ai, privacy, kids, intelligence, unethically,...","[[""I mean, I understand that. So going off of ..."
6,5,14,5_academia_ai_grad_tech,"[academia, ai, grad, tech, studying, career, p...","[[""Yeah. So pretty much been an academic in tr..."
7,6,9,6_bias_ai_biased_learning,"[bias, ai, biased, learning, data, research, t...","[[""Yeah. You look at AI used to make decisions..."
8,7,9,7_consumer_teaching_do_about,"[consumer, teaching, do, about, things, what, ...",[['Yeah. Absolutely. You answered everything. ...


In [36]:
key_topic_model.visualize_barchart()

In [37]:
key_topic_model.visualize_documents(docs_clo)

# MMR

In [38]:
# Fine tuning the model with MMR 
mmr_representation_model = MaximalMarginalRelevance(diversity=0.3)
mmr_topic_model = BERTopic(representation_model=mmr_representation_model,umap_model=umap_model, hdbscan_model=hdbscan_model)

In [39]:
# run and fit MMR
mmr_topics, mmr_probs = mmr_topic_model.fit_transform(docs_clo)

In [40]:
# Frequent topic using MMR
mmr_topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,86,-1_so_with_do_its,"[so, with, do, its, they, about, think, like, ...","[['Yeah. I mean, very much the way I would han..."
1,0,27,0_ai_human_we_lot,"[ai, human, we, lot, be, you, really, team, da...","[[""Yeah. And even just the tendency, like one ..."
2,1,27,1_in_data_ai_so,"[in, data, ai, so, be, are, algorithm, think, ...","[[""Yeah. You look at AI used to make decisions..."
3,2,27,2_to_ai_we_people,"[to, ai, we, people, about, you, because, thin...","[['Okay, here\'s how I\'m going to think about..."
4,3,23,3_think_it_start_like,"[think, it, start, like, understand, should, y...","[[""I think to not be fearful of it, but to und..."
5,4,13,4_that_information_kids_think,"[that, information, kids, think, like, learnin...","[[""I mean, I understand that. So going off of ..."
6,5,13,5_to_robot_we_work,"[to, robot, we, work, needs, be, do, say, spee...",[['A bit. Yeah. So the primary ones that I\'d ...
7,6,13,6_computer_so_science_ai,"[computer, so, science, ai, at, school, tech, ...","[[""Yeah. So pretty much been an academic in tr..."
8,7,10,7_think_alexa_kids_she,"[think, alexa, kids, she, have, learn, logic, ...","[[""For kids, especially younger, the more tact..."
9,8,9,8_to_about_like_they,"[to, about, like, they, what, thats, netflix, ...",[['Yeah. Absolutely. You answered everything. ...


In [41]:
mmr_topic_model.visualize_barchart()

In [42]:
mmr_topic_model.visualize_documents(docs_clo)